# Import Library

In [1]:
# Install library yang dibutuhkan untuk scraping
!pip install pandas google-play-scraper -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00


In [2]:
# Hanya import library yang relevan untuk tahap scraping
import pandas as pd
from google_play_scraper import reviews, Sort
import time

print("Library untuk scraping siap digunakan.")

Library untuk scraping siap digunakan.


# Konfigurasi Scraping

In [3]:
# ID Aplikasi yang akan di-scrape
app_id = 'com.lazada.android'

# Jumlah ulasan yang ingin diambil.
# Menggunakan reviews() agar parameter 'count' berfungsi dengan baik.
review_count = 40000

print(f"Memulai proses scraping untuk aplikasi '{app_id}'...")
start_time = time.time()

# Melakukan scraping dengan google_play_scraper
# Menggunakan 'reviews' bukan 'reviews_all' untuk mengambil jumlah spesifik
result, continuation_token = reviews(
    app_id,
    lang='id',                # Bahasa ulasan: Indonesia
    country='id',             # Negara: Indonesia
    sort=Sort.MOST_RELEVANT,  # Urutkan berdasarkan ulasan paling relevan
    count=review_count,       # Jumlah ulasan yang ingin diambil
    filter_score_with=None    # Ambil semua skor rating (1 sampai 5)
)

end_time = time.time()
print(f"Proses scraping selesai dalam {end_time - start_time:.2f} detik.")

# Jika hasil scraping kosong, hentikan proses
if not result:
    print("Tidak ada ulasan yang berhasil di-scrape. Proses dihentikan.")
else:
    # --- Konversi ke DataFrame dan Penyimpanan Data Mentah ---

    # Konversi hasil scraping (yang berupa list of dictionaries) ke DataFrame Pandas
    df_reviews_raw = pd.DataFrame(result)

    print(f"\nBerhasil mendapatkan {len(df_reviews_raw)} ulasan.")
    print("Contoh 5 baris data mentah:")
    display(df_reviews_raw.head())

    # Menyimpan DataFrame mentah ke file CSV.
    # Cara ini jauh lebih ringkas dan efisien daripada menggunakan loop dan csv.writer.
    raw_filename = 'ulasan_aplikasi_mentah.csv'
    df_reviews_raw.to_csv(raw_filename, index=False)
    print(f"\nData ulasan mentah berhasil disimpan ke '{raw_filename}'")


    # --- Pembersihan Data Awal ---
    print("\nMemulai proses pembersihan data awal...")

    # 1. Pilih hanya kolom yang relevan untuk analisis sentimen
    # Kita hanya membutuhkan 'content' untuk analisis, tapi menyimpan kolom lain bisa berguna
    df_clean = df_reviews_raw[['userName', 'content', 'score', 'at']].copy()

    # 2. Hapus baris di mana kolom 'content' (ulasan) kosong/NaN
    df_clean.dropna(subset=['content'], inplace=True)

    # 3. Hapus ulasan yang duplikat berdasarkan kontennya
    df_clean.drop_duplicates(subset=['content'], keep='first', inplace=True)

    # 4. Konversi kolom tanggal 'at' menjadi format datetime
    df_clean['at'] = pd.to_datetime(df_clean['at'])

    print(f"Jumlah ulasan setelah dibersihkan (dihapus NaN dan duplikat): {len(df_clean)}")
    print("Info DataFrame setelah pembersihan:")
    df_clean.info()

    # Menyimpan DataFrame yang sudah bersih untuk tahap selanjutnya
    clean_filename = 'ulasan_aplikasi_clean.csv'
    df_clean.to_csv(clean_filename, index=False)
    print(f"\nData ulasan bersih siap untuk dianalisis dan disimpan ke '{clean_filename}'")

Memulai proses scraping untuk aplikasi 'com.lazada.android'...
Proses scraping selesai dalam 22.14 detik.

Berhasil mendapatkan 40000 ulasan.
Contoh 5 baris data mentah:


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,5e546f3c-07b8-48a2-b6bd-06eb87328b8d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Gimana mau belanja, update terus, tapi buruk k...",1,1,5.12,2018-09-23 20:08:52,Mohon maaf atas kendala teknis yang Anda alami...,2018-09-23 23:10:55,5.12
1,943f8ca0-3e3d-40e5-bd83-fb36ed812e2d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"pengiriman semakin gak jelas, sudah brp kali i...",1,69,7.76.2,2025-06-03 05:56:08,Kami sangat menghargai umpan balik kakak menge...,2025-06-03 05:58:29,7.76.2
2,a9441a28-98d8-47a0-99e6-09ec4221139a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"lazada sekarang kok jadi nyebelin ,, bukannya ...",2,39,7.76.2,2025-06-03 07:26:21,Kami sangat menghargai masukan Kakak mengenai ...,2025-06-03 07:48:21,7.76.2
3,c81f1d9d-4848-4ac1-8595-880785db1e17,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,menurut saya sih ya lazada ini bagus banyak di...,1,14,7.76.2,2025-06-02 22:09:34,Kami menghargai feedback Kakak dan menyadari p...,2025-06-02 22:17:29,7.76.2
4,7626e358-da73-4f3f-97ab-07acd0f0c0f3,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,tenyata banyak pedagang/ toko yg menjual baran...,4,24,7.76.2,2025-06-02 13:49:54,Kami sangat menghargai rating yang baik yang d...,2025-06-02 13:59:03,7.76.2



Data ulasan mentah berhasil disimpan ke 'ulasan_aplikasi_mentah.csv'

Memulai proses pembersihan data awal...
Jumlah ulasan setelah dibersihkan (dihapus NaN dan duplikat): 39848
Info DataFrame setelah pembersihan:
<class 'pandas.core.frame.DataFrame'>
Index: 39848 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   userName  39848 non-null  object        
 1   content   39848 non-null  object        
 2   score     39848 non-null  int64         
 3   at        39848 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.5+ MB

Data ulasan bersih siap untuk dianalisis dan disimpan ke 'ulasan_aplikasi_clean.csv'
